In [ ]:
pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta

# Análise Exploratória

## Tratamento de dados

In [ ]:
def is_leap_year(year):
    return calendar.isleap(year)

In [ ]:
def num_rows(df):
  return len(df)

In [ ]:
def num_columns(df):
  return len(df.columns)

In [ ]:
def date_and_time(df):
  df['Data'] = pd.to_datetime(df['Data'])

  df['Date'] = df['Data'].dt.date
  df['Time'] = df['Data'].dt.time

In [ ]:
def duplicates(df):
  return df['Data'].duplicated()

In [ ]:
def mean_time_difference(df):
  return (time_diff(df)).mean()

In [ ]:
def time_diff(df):
  time_diff = df['Data'].diff()
  return time_diff[time_diff > pd.Timedelta(0)]

In [ ]:
def mean_time_difference_per_year(df):
  df_copy = df.copy()
  df_copy['TimeDiff'] = df_copy['Data'].diff()
  df_copy['TimeDiff'] = df_copy['TimeDiff'][df_copy['TimeDiff'] > pd.Timedelta(0)]

  df_copy['Year'] = df_copy['Data'].dt.year

  return df_copy.groupby('Year')['TimeDiff'].mean()


In [ ]:
def mean_time_difference_per_month(df):
  df_copy = df.copy()

  df_copy['TimeDiff'] = df['Data'].diff()

  df_copy['TimeDiff'] = df_copy['TimeDiff'][df_copy['TimeDiff'] > pd.Timedelta(0)]

  df_copy['Year'] = df_copy['Data'].dt.year
  df_copy['Month'] = df_copy['Data'].dt.month

  return df_copy.groupby(['Year', 'Month'])['TimeDiff'].mean()

In [ ]:
def format(df):
  formato = ['Date', 'Time', 'Caudal']
  return df[formato]

In [ ]:
def has_seconds(df):
  df['Time'] = df['Time'].apply(lambda x: x.strftime('%H:%M:%S'))
  return any(':00' not in str_time for str_time in df['Time'])

In [ ]:
def num_unique_dates(df):
  return df['Date'].nunique()

In [ ]:
def date_datetime(df):
  df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
def unique_years(df):
  return df['Date'].dt.year.unique()

In [ ]:
def is_ordered_ascending(df):
  return df['Date'].is_monotonic_increasing

In [ ]:
def date_date_format(df):
  df['Date'] = df['Date'].dt.date

In [ ]:
def sort_date(df):
  return df.sort_values(by='Date')

In [ ]:
def year_month_day(df):
  df['Year'] = df['Date'].dt.year
  df['Month'] = df['Date'].dt.month
  df['Day'] = df['Date'].dt.day

  return df

In [ ]:
def average_measurements_per_day(df):
  return df.groupby('Date').size().mean()

In [ ]:
def average_measurements_per_year(df):
  return df.groupby(['Year', 'Date']).size().groupby('Year').mean()

In [ ]:
def average_measurements_per_year_month(df):
   return df.groupby(['Year', 'Month', 'Date']).size().groupby(['Year', 'Month']).mean()

In [ ]:
def measurements_per_year_month_boxplot(df):
  daily_measurements_count = df.groupby(['Year', 'Month', 'Date']).size()

  plot_data = daily_measurements_count.reset_index(name='Measurements')

  for year in unique_years(df):
      year_data = plot_data[plot_data['Year'] == year]

      plt.figure(figsize=(8, 6))
      sns.boxplot(x='Month', y='Measurements', data=year_data, whis=3, palette="Paired")
      plt.title(f'Número de medições diárias em {year}')
      plt.xlabel('Month')
      plt.ylabel('Número de medições')
      plt.show()


In [ ]:
def unique_month_counts(df):
  return df.drop_duplicates(subset=['Year', 'Month']).groupby('Year').size().reset_index(name='Número de meses')

In [ ]:
def unique_days_counts(df):
  return df.drop_duplicates(subset=['Year', 'Month','Day']).groupby('Year').size().reset_index(name='Número de dias')

In [ ]:
def expected_days_print(df):
  for year, num_days in zip((unique_days_counts(df))['Year'], (unique_days_counts(df))['Número de dias']):

    expected_days = 366 if pd.Timestamp(f'{year}-12-31').is_leap_year else 365
    expected_days -= (12 - len(df[df['Year'] == year]['Month'].unique())) * 30

    print(f"Existem medições de {num_days} dias distintos em {year}. Esperava-se que existissem {expected_days}.")

In [ ]:
def caudal_values_chart(df):
    plt.figure(figsize=(10, 6))

    for year in unique_years(df):
        year_data = df[df['Year'] == year]
        plt.plot(year_data['Date'], year_data['Caudal'], label=str(year))

    plt.xlabel('Data')
    plt.ylabel('Caudal')
    plt.title('Valores de Caudal')
    plt.legend(title='Ano')
    plt.show()


In [ ]:
def caudal_values_chart_yearly(df):
  for year in unique_years(df):
    year_data = df[df['Year'] == year]

    plt.figure(figsize=(10, 6))
    plt.plot(year_data['Date'], year_data['Caudal'])
    plt.xlabel('Data')
    plt.ylabel('Caudal')
    plt.title(f'Valores de Caudal em {year}')
    plt.show()

In [ ]:
def time_datetime(df):
  df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

In [ ]:
def invalid_time_values(df):
  return df[df['Time'].isna()]

In [ ]:
def time_time(df):
  df['Time'] = df['Time'].dt.time

In [ ]:
def correct_caudal(df):
  df['Caudal'] = pd.to_numeric(df['Caudal'], errors='coerce')

In [ ]:
def fails(df):
  return df[df['Caudal'].isna()]

In [ ]:
def total_fails(df):
  return len(fails(df))

In [ ]:
def non_integer_counts(df):
  non_integer_mask = df['Caudal'].isna()
  return df[non_integer_mask].groupby('Year').size().reset_index(name='Número de Falhas')

In [ ]:
def days_with_no_data(df):
  return df.groupby('Date').filter(lambda x: x['Caudal'].notna().any()).groupby('Date').filter(lambda x: x['Caudal'].isna().all())['Date'].unique()

In [ ]:
def entrys_per_year(df):
  return df.groupby('Year')['Caudal'].count().reset_index(name='Número Total de medições')

In [ ]:
def entries_without_errors_per_year(entrys_per_year_df, non_integer_counts_df):
  entries_without_fails_per_year = entrys_per_year_df.copy()
  entries_without_fails_per_year['Medições sem Falhas'] = entrys_per_year_df['Número Total de medições'] - non_integer_counts_df['Número de Falhas']
  return entries_without_fails_per_year

In [ ]:
def entries_without_errors_per_year_percent(df):
  df['Medições sem falhas (%)'] = ((df['Medições sem Falhas'] / df['Número Total de medições']) * 100)

  entries_without_errors_per_year_percent = df.copy()
  entries_without_errors_per_year_percent.drop('Medições sem Falhas', axis=1, inplace=True)

  return entries_without_errors_per_year_percent

In [ ]:
def entries_with_errors_per_year_percent(df):
  entries_with_errors_per_year_percent = df.copy()

  entries_with_errors_per_year_percent['Medições com falhas (%)'] = (
    (100 - df['Medições sem falhas (%)'])
  )

  entries_with_errors_per_year_percent.drop('Medições sem falhas (%)', axis=1, inplace=True)

  return entries_with_errors_per_year_percent

In [ ]:
def unique_years_no_error(df):
  return (df_no_errors(df))['Year'].unique()

In [ ]:
def df_no_errors(df):
  return df[~df['Caudal'].isna()]

In [ ]:
def mean_results(df, no_errors_df):
  return no_errors_df.groupby(['Year', 'Month'], as_index=False).agg(
    Average=('Caudal', 'mean'),
  ).reset_index()

In [ ]:
def mean_results_yearly(df, no_errors_df):
  return no_errors_df.groupby('Year', as_index=False).agg(
    Average=('Caudal', 'mean'),
  ).reset_index()

In [ ]:
def means_caudal_yearly(df):

  no_errors_df = df_no_errors(df)

  result = mean_results(df, no_errors_df)

  result_yearly = mean_results_yearly(df, no_errors_df)

  for year in unique_years_no_error(df):
    year_data = result[result['Year'] == year]

    plt.figure(figsize=(10, 6))
    plt.plot(year_data['Month'] , year_data['Average'], label='Média', marker='o', linestyle='-')

    plt.axhline(result_yearly.loc[result_yearly['Year'] == year, 'Average'].values[0], linestyle='dashed', color='blue',
                label= f'Média anual : {result_yearly.loc[result_yearly["Year"] == year, "Average"].values[0]:.2f}')

    for i, avg in enumerate(year_data['Average']):
      plt.text(year_data['Month'].iloc[i] , avg, f'{avg:.2f}', ha='center', va='bottom', color='blue')

    plt.xlabel('Mês')
    plt.ylabel('Caudal')
    plt.title(f'Média das medições de Caudal do ano {year}')
    plt.xticks(range(1, 13), [str(month) for month in range(1, 13)])
    plt.legend()
    plt.show()

In [ ]:
def means_caudal(df):

  no_errors_df = df_no_errors(df)

  result = mean_results(df, no_errors_df)

  result_yearly = mean_results_yearly(df, no_errors_df)

  plt.figure(figsize=(10, 6))
  for year in unique_years_no_error(df):
    year_data = result[result['Year'] == year]
    plt.plot(year_data['Month'], year_data['Average'], label=f'Ano {year}')

  overall_monthly_average = result.groupby('Month')['Average'].mean()

  plt.plot(overall_monthly_average.index, overall_monthly_average.values, label='Média Geral', linestyle='--', color='black')

  plt.xlabel('Mês')
  plt.ylabel('Caudal')
  plt.title('Médias de medições de Caudal')
  plt.xticks(range(1, 13), [str(month) for month in range(1, 13)])
  plt.legend()
  plt.show()

In [ ]:
def std_monthly(df):
  return df.groupby(['Year', 'Month'], as_index=False).agg(
    Desvio_Padrao=('Caudal', 'std')
).reset_index()

In [ ]:
def std_yearly(df):
  return df.groupby('Year', as_index=False).agg(
    Desvio_Padrao=('Caudal', 'std')
).reset_index()


In [ ]:
def caudal_boxplots(df):
  for year in unique_years_no_error(df):
    plt.figure(figsize=(12, 8))
    year_data = df[df['Year'] == year]

    ax = sns.boxplot(x='Month', y='Caudal', data=year_data, whis=3, palette="Set3")

    plt.title(f'Boxplot das medições de Caudal em {year}')
    plt.xlabel('Mês')
    plt.ylabel('Medição do Caudal')
    plt.show()

In [ ]:
def caudal_statistics(df_no_errors):
  dfs = []

  for year in df_no_errors['Year'].unique():
    for month in df_no_errors['Month'].unique():
        subset_data = df_no_errors[(df_no_errors['Year'] == year) & (df_no_errors['Month'] == month)]

        if not subset_data.empty:
            min_value = subset_data['Caudal'].min()
            q1_value = np.percentile(subset_data['Caudal'], 25)
            median_value = np.median(subset_data['Caudal'])
            q3_value = np.percentile(subset_data['Caudal'], 75)
            max_value = subset_data['Caudal'].max()

            dfs.append(pd.DataFrame({
                'Year': [year],
                'Month': [month],
                'Min_Value': [min_value],
                'Q1_Value': [q1_value],
                'Median': [median_value],
                'Q3_Value': [q3_value],
                'Max_Value': [max_value]
            }))

  summary_df = pd.concat(dfs, ignore_index=True)

  summary_df_sorted = summary_df.sort_values(by=['Year', 'Month'])
  return summary_df_sorted

In [ ]:
def resultados(df):

  print(f"Existem {num_rows(df)} medições de Caudal. \n")

  print(f"A DataFrame tem {num_columns(df)} colunas \n")

  date_and_time(df)

  if (duplicates(df)).any():
    print("Linhas duplicadas:")
    print(df[duplicates])
    df = df.drop_duplicates(subset=['Data'])
  else:
    print("Não existem duplicados.\n")

  print("Média de tempo entre medições:", mean_time_difference(df), "\n")

  print("Média de tempo entre medições por ano:")
  print(mean_time_difference_per_year(df))

  print("\nMédia de tempo entre medições por ano e mês:")
  print(mean_time_difference_per_month(df))


  df.drop('Data', axis=1, inplace=True)

  df = format(df)

  if has_seconds(df):
    subset = df[df['Time'].apply(lambda x: ':00' not in x)]
    print("\n Os dados com valor nos segundos:")
    print(subset)
  else:
    print("\n O valor dos segundos está sempre a 00.\n")


  print(f"Existem medições de {num_unique_dates(df)} dias distintos.\n")

  date_datetime(df)

  print("Temos dados dos seguintes anos:")
  print(unique_years(df))

  if is_ordered_ascending(df):
    print("\n Os dados estão organizados de forma cronológica.\n")
  else:
    print("\n Os dados não estão organizados de forma cronológica.\n")
    df = sort_date(df)

  date_date_format(df)
  print(f"A primeira medição foi feita em: {df['Date'][0]}\n")
  print(f"A ultima medição foi feita em: {df['Date'][len(df)-1]}\n")

  date_datetime(df)
  df = year_month_day(df)

  print(f"Valor médio de medições diárias: {average_measurements_per_day(df)} \n")

  print("Valor médio de medições diárias por ano:")
  print(average_measurements_per_year(df))

  print("\nValor médio de medições diárias por ano e mês:")
  print(average_measurements_per_year_month(df))

  measurements_per_year_month_boxplot(df)

  print("\n Quantos meses tiveram medições naquele ano:")
  print(unique_month_counts(df))

  print("\n Quantos dias tiveram medições naquele ano:")
  print(unique_days_counts(df))
  print('\n')

  expected_days_print(df)

  caudal_values_chart(df)
  caudal_values_chart_yearly(df)

  time_datetime(df)

  if not invalid_time_values(df).empty:
    print("\n Existem falhas no tempo das seguintes leituras:")
    print(invalid_time_values(df))
  else:
    print("\n Não existem falhas nos tempos")

  time_time(df)

  if not fails(df).empty:
    print("\n Existem as seguintes falhas nos dados:")
    print(format(fails(df)))
  else:
    print("\n Não existem falhas nos dados.")

  print(f"\n Existem {total_fails(df)} falhas de leituras \n")

  correct_caudal(df)

  fails_count = non_integer_counts(df)
  print("Número de falhas por ano:")
  print(fails_count)

  days_without_data = days_with_no_data(df)
  if len(days_without_data) > 0:
    print("\n Existem dias inteiros sem valores.")
    print("Dias sem valores:", days_without_data)
  else:
    print("\n Todos os dias tem pelo menos uma medição sem falhas.")

  entrys_year = entrys_per_year(df)
  print('\n Número de medições por ano:')
  print(entrys_year)

  entries_without_fails_per_year = entries_without_errors_per_year(entrys_year, fails_count)
  print("\n Número de medições sem falhas:")
  print(entries_without_fails_per_year)

  entries_without_fails_per_year_percent = entries_without_errors_per_year_percent(entries_without_fails_per_year)
  print("\n Percentagem de medições sem falhas:")
  print(entries_without_fails_per_year_percent)

  print("\n Percentagem de medições com falhas:")
  print(entries_with_errors_per_year_percent(entries_without_fails_per_year_percent))

  print('\n')
  means_caudal_yearly(df)
  means_caudal(df)

  print("\n", (std_monthly(df_no_errors(df)))[['Year', 'Month', 'Desvio_Padrao']])
  print("\n", (std_yearly(df_no_errors(df)))[['Year', 'Desvio_Padrao']])

  print('\n')
  caudal_boxplots(df)
  print('\n')
  print(caudal_statistics(df_no_errors(df)))

## Unico Output

### Alcogulhe

In [ ]:
alcogulhe = pd.read_excel("/content/alcogulhe.xlsx")

resultados(alcogulhe)

### Arnal

In [ ]:
arnal = pd.read_excel("/content/arnal.xlsx")

resultados(arnal)

### Cerca

In [ ]:
cerca = pd.read_excel("/content/cerca.xlsx")

resultados(cerca)

### Maceirinha

In [ ]:
maceirinha = pd.read_excel("/content/maceirinha.xlsx")

resultados(maceirinha)

### Porto Carro

In [ ]:
porto_carro = pd.read_excel("/content/porto_carro.xlsx")

resultados(porto_carro)

# Normalização

In [ ]:
df = pd.read_excel("/content/.xlsx")

##Funcões

In [ ]:
def normalize(resampled_df, time):
    for col in resampled_df.columns:

        missing_mask = resampled_df[col].isnull()

        for idx, value in resampled_df[col][missing_mask].items():

            before_idx = None
            for i in reversed(range(resampled_df.index.get_loc(idx))):
                if not pd.isnull(resampled_df.at[resampled_df.index[i], col]):
                    before_idx = resampled_df.index[i]
                    break
            after_idx = None
            for i in range(resampled_df.index.get_loc(idx), len(resampled_df.index)):
                if not pd.isnull(resampled_df.at[resampled_df.index[i], col]):
                    after_idx = resampled_df.index[i]
                    break

            if before_idx is not None and after_idx is not None and (after_idx - before_idx).total_seconds() <= pd.Timedelta(minutes=time).total_seconds():
                resampled_df.at[idx, col] = resampled_df[col][before_idx] + \
                                            ((resampled_df[col][after_idx] - resampled_df[col][before_idx]) /
                                            (after_idx - before_idx).total_seconds()) * \
                                            (idx - before_idx).total_seconds()

In [ ]:
def years(df):
  return df.index.year.unique()

def normalizedPlot(df, time):

  plt.figure(figsize=(10, 6))


  for year in years(df):

      year_data = df[df.index.year == year]

      plt.plot(year_data.index, year_data['Caudal'], label=str(year))

  plt.xlabel('Data')
  plt.ylabel('Caudal')
  plt.title(f'Valores normalizados com intervalo máximo de {time} minutos')
  plt.legend(title='Ano')

  plt.ylim(-5, 90)
  plt.show()


In [ ]:
def calculate_mean_area(df):
    mean_areas = {}
    for date, data in df.groupby(df.index.date):
        times = data.index.time
        values = data['Caudal']
        areas = []
        for i in range(len(values) - 1):
            if not pd.isnull(values[i]) and not pd.isnull(values[i + 1]):
                x = np.array([times[i].hour * 3600 + times[i].minute * 60 + times[i].second,
                              times[i + 1].hour * 3600 + times[i + 1].minute * 60 + times[i + 1].second])
                y = np.array([values[i], values[i + 1]])
                areas.append(np.trapz(y, x))
        if areas:
            mean_areas[date] = np.mean(areas)
    return mean_areas

In [ ]:
def diference(df, column1, column2):
  plt.figure(figsize=(10, 6))
  plt.plot(df['Date'], df[f'{column1}'] - df[f'{column2}'], marker='o')
  plt.xlabel('Date')
  plt.ylabel('Difference in Area')
  plt.title('Difference in Daily Mean Area between Old and New DataFrames')
  plt.xticks(rotation=45)
  plt.grid(True)
  plt.tight_layout()
  plt.show()

In [ ]:
def comparison(df, column1, column2):
  df.plot(kind='scatter', x=column1, y=column2, s=32, alpha=.8)
  plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
def remove_outliers_iqr(df, column):
    df['Year'] = df['Data'].dt.year
    df['Month'] = df['Data'].dt.month
    grouped = df.groupby(['Year', 'Month'])

    df_filtered = pd.DataFrame()
    outliers_above = pd.DataFrame()
    outliers_below = pd.DataFrame()

    for (year, month), group in grouped:
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR

        group_filtered = group[(group[column] >= lower_bound) & (group[column] <= upper_bound)]
        outliers_above_group = group[group[column] > upper_bound]
        outliers_below_group = group[group[column] < lower_bound]

        df_filtered = pd.concat([df_filtered, group_filtered])
        outliers_above = pd.concat([outliers_above, outliers_above_group])
        outliers_below = pd.concat([outliers_below, outliers_below_group])

    return df_filtered, outliers_above, outliers_below


In [ ]:
def boxplot_outliers(df):
  df['Data'] = pd.to_datetime(df['Data'])

  plt.figure(figsize=(10, 6))
  plt.boxplot(df['Caudal'], whis=100)
  plt.title('Diagrama de Caixas dos Outliers')
  plt.ylabel('Caudal')
  plt.xticks([1], ['Caudal'])
  plt.grid(True)
  plt.show()

In [ ]:
def statistics(df_no_errors):
    dfs = []

    df_no_errors['Year'] = df_no_errors['Data'].dt.year
    df_no_errors['Month'] = df_no_errors['Data'].dt.month

    for year in df_no_errors['Year'].unique():
        for month in df_no_errors['Month'].unique():
            subset_data = df_no_errors[(df_no_errors['Year'] == year) & (df_no_errors['Month'] == month)]

            if not subset_data.empty:
                min_value = subset_data['Caudal'].min()
                max_value = subset_data['Caudal'].max()
                count = len(subset_data)

                dfs.append(pd.DataFrame({
                    'Year': [year],
                    'Month': [month],
                    'Min_Value': [min_value],
                    'Max_Value': [max_value],
                    'Count': [count]
                }))

    summary_df = pd.concat(dfs, ignore_index=True)

    summary_df_sorted = summary_df.sort_values(by=['Year', 'Month'])
    return summary_df_sorted

In [ ]:
def missings_study(df):
  df['TimeDiff'] = df['Data'].diff()

  group_start = df.iloc[0]['Data']
  group_count = 1

  result_dates = []
  result_consecutive_points = []
  result_time = []

  for i in range(1, len(df)):
      if df.iloc[i]['Data'] - df.iloc[i - 1]['Data'] > pd.Timedelta(minutes=15):
          result_dates.append(group_start)
          result_consecutive_points.append(group_count)
          duration_minutes = group_count * 15
          days = duration_minutes // (24 * 60)
          remaining_minutes = duration_minutes % (24 * 60)
          hours = remaining_minutes // 60
          minutes = remaining_minutes % 60
          time_str = f"{days} days {hours:02d}:{minutes:02d}:00"
          result_time.append(time_str)
          group_start = df.iloc[i]['Data']
          group_count = 1
      else:
          group_count += 1

  result_dates.append(group_start)
  result_consecutive_points.append(group_count)
  duration_minutes = (group_count - 1) * 15
  days = duration_minutes // (24 * 60)
  remaining_minutes = duration_minutes % (24 * 60)
  hours = remaining_minutes // 60
  minutes = remaining_minutes % 60
  time_str = f"{days} days {hours:02d}:{minutes:02d}:00"
  result_time.append(time_str)

  return pd.DataFrame({'Data': result_dates, 'Consecutive_Points': result_consecutive_points, 'Time': result_time})

In [ ]:
def convert_to_time(value):
    minutes = value * 15
    hours = minutes // 60
    minutes = minutes % 60
    days = hours // 24
    hours = hours % 24
    seconds = 0
    return '{} days {:02d}:{:02d}:{:02d}'.format(int(days), int(hours), int(minutes), int(seconds))

In [ ]:
def convert_to_time_less_than_1day(value):
    minutes = value * 15
    hours = minutes // 60
    minutes = minutes % 60
    seconds = 0
    return '{:02d}:{:02d}:{:02d}'.format(int(hours), int(minutes), int(seconds))

## Com outliers

In [ ]:
copy_df = df.copy()

In [ ]:
copy_df['Data'] = pd.to_datetime(df['Data'])

In [ ]:
copy_df.set_index('Data', inplace=True)

In [ ]:
resampled_df = copy_df.resample('15T').asfreq()

In [ ]:
max30min_df = resampled_df.copy()
max15min_df = resampled_df.copy()
max60min_df = resampled_df.copy()

In [ ]:
normalize(max30min_df, 30)
normalize(max15min_df, 15)
normalize(max60min_df, 60)

In [ ]:
normalizedPlot(max30min_df, 30)

In [ ]:
normalizedPlot(max15min_df, 15)

In [ ]:
normalizedPlot(max60min_df, 60)

In [ ]:
original_areas = calculate_mean_area(copy_df)

In [ ]:
original_areas = calculate_mean_area(copy_df)
max30min_areas = calculate_mean_area(max30min_df)
max15min_areas = calculate_mean_area(max15min_df)
max60min_areas = calculate_mean_area(max60min_df)

original_areas_df = pd.DataFrame(list(original_areas.items()), columns=['Date', 'Original_Area'])
max30min_areas_df = pd.DataFrame(list(max30min_areas.items()), columns=['Date', 'New_Area'])
max15min_areas_df = pd.DataFrame(list(max15min_areas.items()), columns=['Date', 'New_Area'])
max60min_areas_df = pd.DataFrame(list(max60min_areas.items()), columns=['Date', 'New_Area'])

comparison_max30min_df = original_areas_df.merge(max30min_areas_df, on='Date', how='outer')
comparison_max15min_df = original_areas_df.merge(max15min_areas_df, on='Date', how='outer')
comparison_max60min_df = original_areas_df.merge(max60min_areas_df, on='Date', how='outer')

In [ ]:
diference(comparison_max30min_df, 'Original_Area', 'New_Area')

In [ ]:
diference(comparison_max15min_df, 'Original_Area', 'New_Area')

In [ ]:
diference(comparison_max60min_df, 'Original_Area', 'New_Area')

In [ ]:
max30min_vs_max15min= comparison_max30min_df.merge(comparison_max15min_df, on='Date', how='outer')
max30min_vs_max60min = comparison_max30min_df.merge(comparison_max60min_df, on='Date', how='outer')
max15min_vs_max60min = comparison_max15min_df.merge(comparison_max60min_df, on='Date', how='outer')

In [ ]:
comparison(max30min_vs_max15min, 'New_Area_x', 'New_Area_y')

In [ ]:
comparison(max30min_vs_max60min, 'New_Area_x', 'New_Area_y')

In [ ]:
comparison(max15min_vs_max60min, 'New_Area_x', 'New_Area_y')

## Sem Outliers

In [ ]:
no_outliers = df.copy()
no_outliers, outliers_above, outliers_below = remove_outliers_iqr(no_outliers, 'Caudal')

In [ ]:
original_outlier_above = outliers_above
original_outlier_below = outliers_below

In [ ]:
missings_df = df[df['Caudal'].isna()].copy()
print(statistics(missings_df).drop(['Min_Value', 'Max_Value'], axis=1))

print(f"\nExistem {len(missings_df)} valores omissos o que corresponde a  {(len(missings_df)*100)/(len(no_outliers) + len(missings_df))}% da amostra")

In [ ]:
len(outliers_above)

In [ ]:
len(outliers_below)

In [ ]:
if (len(outliers_above) > 0):
  boxplot_outliers(outliers_above)

In [ ]:
if (len(outliers_above) > 0):
  print(statistics(outliers_above))

print(f"\nforam retirados {len(outliers_above)} valores, o que corresponde a {(len(outliers_above)*100)/(len(no_outliers) + len(df[df['Caudal'].isna()]))}% da amostra")

In [ ]:
outliers_above

In [ ]:
if (len(outliers_below) > 0):
  boxplot_outliers(outliers_below)

In [ ]:
if (len(outliers_below) > 0):
  print(statistics(outliers_below))

print(f"\nforam retirados {len(outliers_below)} valores, o que corresponde a {(len(outliers_below)*100)/(len(no_outliers) + len(df[df['Caudal'].isna()]))}% da amostra")

In [ ]:
no_outliers['Data'] = pd.to_datetime(df['Data'])

In [ ]:
no_outliers.set_index('Data', inplace=True)

In [ ]:
resampled_df = no_outliers.resample('15T').asfreq()

closest_before = np.empty(len(resampled_df))
closest_after = np.empty(len(resampled_df))
data_before = [pd.NaT] * len(resampled_df)
data_after = [pd.NaT] * len(resampled_df)

no_outliers_idx = no_outliers.index.values
no_outliers_caudal = no_outliers['Caudal'].values

total_rows = len(resampled_df)
for idx, (i, row) in enumerate(resampled_df.iterrows()):
    if pd.isnull(row['Caudal']):
        idx_before = np.where(no_outliers_idx < i)[0]
        if len(idx_before) > 0:
            closest_before[idx] = no_outliers_caudal[idx_before[-1]]
            data_before[idx] = no_outliers_idx[idx_before[-1]]
        else:
            closest_before[idx] = np.nan

        idx_after = np.where(no_outliers_idx > i)[0]
        if len(idx_after) > 0:
            closest_after[idx] = no_outliers_caudal[idx_after[0]]
            data_after[idx] = no_outliers_idx[idx_after[0]]
        else:
            closest_after[idx] = np.nan
    else:
        closest_before[idx] = row['Caudal']
        closest_after[idx] = row['Caudal']
        data_before[idx] = i
        data_after[idx] = i

    if idx % (total_rows // 20) == 0:
        print(f"{int((idx / total_rows) * 100)}% done")

closest_df = pd.DataFrame({
    'Data_before': data_before,
    'Closest_before': closest_before,
    'Data_after': data_after,
    'Closest_after': closest_after
}, index=resampled_df.index)


In [ ]:
closest_df

In [ ]:
closest_df['time_gap'] = (closest_df['Data_after'] - closest_df['Data_before']).dt.total_seconds() <= 30 * 60

In [ ]:
closest_df

In [ ]:
def calculate_caudal(row):
    if row['time_gap']:
        x = row.name
        x1 = row['Data_before']
        x2 = row['Data_after']
        y1 = row['Closest_before']
        y2 = row['Closest_after']
        time_diff = (x2 - x1).total_seconds()
        if time_diff == 0:
            return y1
        else:
            caudal = y1 + ((x - x1).total_seconds() * (y2 - y1) / time_diff)
            return caudal
    else:
        return None

closest_df['Caudal'] = closest_df.apply(calculate_caudal, axis=1)

In [ ]:
resampled_df['Caudal'] = closest_df['Caudal']

In [ ]:
resampled_df

In [ ]:
missings_normalized = resampled_df[resampled_df['Caudal'].isna()].copy()
missings_normalized.reset_index(inplace=True)
print(statistics(missings_normalized).drop(['Min_Value', 'Max_Value'], axis=1))

print(f"\nExistem {len(missings_normalized)} valores omissos o que corresponde a  {(len(missings_normalized)*100)/len(resampled_df)}% da amostra")

In [ ]:
max30min_df = resampled_df.copy()
max15min_df = resampled_df.copy()
max60min_df = resampled_df.copy()

In [ ]:
normalizedPlot(max30min_df, 30)

In [ ]:
normalizedPlot(max15min_df, 15)

In [ ]:
normalizedPlot(max60min_df, 60)

In [ ]:
nan_caudal_df_30 = max30min_df[max30min_df['Caudal'].isna()].copy()
nan_caudal_df_15 = max15min_df[max15min_df['Caudal'].isna()].copy()
nan_caudal_df_60 = max60min_df[max60min_df['Caudal'].isna()].copy()

In [ ]:
value_caudal_df_30 = max30min_df[max30min_df['Caudal'].notna()].copy()
#NOT BEING USED

In [ ]:
nan_caudal_df_30.reset_index(inplace=True)
nan_caudal_df_15.reset_index(inplace=True)
nan_caudal_df_60.reset_index(inplace=True)

In [ ]:
missings_gap30_df = missings_study(nan_caudal_df_30)

In [ ]:
missings_gap15_df = missings_study(nan_caudal_df_15)

In [ ]:
missings_gap60_df = missings_study(nan_caudal_df_60)

96 = 1 dia <br>
192 = 2 dias

In [ ]:
yticks_values = [0, 50, 100, 150]
yticks_labels = [convert_to_time(y) for y in yticks_values]

sns.boxplot(data=missings_gap30_df, whis=100)
plt.yticks(yticks_values, yticks_labels)
plt.title("Estudo de Falhas pós normalização")
plt.show()

In [ ]:
max_index = missings_gap30_df['Consecutive_Points'].idxmax()
row_with_max_points = missings_gap30_df.loc[max_index]
print(row_with_max_points)

In [ ]:
missings_gap15_df

In [ ]:
yticks_values = [0, 50, 100, 150, 200, 250]
yticks_labels = [convert_to_time(y) for y in yticks_values]

sns.boxplot(data=missings_gap15_df, whis=100)
plt.yticks(yticks_values, yticks_labels)
plt.title("Estudo de Falhas pós normalização com intervalos máximos de 15 min")
plt.show()

In [ ]:
max_index = missings_gap15_df['Consecutive_Points'].idxmax()
row_with_max_points = missings_gap15_df.loc[max_index]
print(row_with_max_points)

In [ ]:
missings_gap60_df

In [ ]:
yticks_values = [0, 50, 100, 150, 200, 250]
yticks_labels = [convert_to_time(y) for y in yticks_values]

sns.boxplot(data=missings_gap60_df, whis=100)
plt.yticks(yticks_values, yticks_labels)
plt.title("Estudo de Falhas pós normalização com intervalos máximos de 60 min")
plt.show()

In [ ]:
max_index = missings_gap60_df['Consecutive_Points'].idxmax()
row_with_max_points = missings_gap60_df.loc[max_index]
print(row_with_max_points)

In [ ]:
above_96_30gap = missings_gap30_df[missings_gap30_df['Consecutive_Points'] > 96]
count_above_96_30gap = above_96_30gap.shape[0]

print("Número de falhas com duração maior que 1 dia:", count_above_96_30gap)
print(f"Isto corresponde a {(count_above_96_30gap*100)/len(missings_gap30_df):.2f}% das falhas")

In [ ]:
above_96_30gap

In [ ]:
above_192_30gap = missings_gap30_df[missings_gap30_df['Consecutive_Points'] > 192]
count_above_192_30gap = above_192_30gap.shape[0]

print("Número de falhas com duração maior que 2 dias:", count_above_192_30gap)
print(f"Isto corresponde a {(count_above_192_30gap*100)/len(missings_gap30_df):.2f}% das falhas")

In [ ]:
above_192_30gap

In [ ]:
above_96_15gap = missings_gap15_df[missings_gap15_df['Consecutive_Points'] > 96]
count_above_96_15gap = above_96_15gap.shape[0]

print("Número de falhas com duração maior que 1 dia:", count_above_96_15gap)
print(f"Isto corresponde a {(count_above_96_15gap*100)/len(missings_gap15_df):.2f}% das falhas")

In [ ]:
above_96_15gap

In [ ]:
above_192_15gap = missings_gap15_df[missings_gap15_df['Consecutive_Points'] > 192]
count_above_192_15gap = above_192_15gap.shape[0]

print("Número de falhas com duração maior que 2 dias:", count_above_192_15gap)
print(f"Isto corresponde a {(count_above_192_15gap*100)/len(missings_gap15_df):.2f}% das falhas")

In [ ]:
above_192_15gap

In [ ]:
above_96_60gap = missings_gap60_df[missings_gap60_df['Consecutive_Points'] > 96]
count_above_96_60gap = above_96_60gap.shape[0]

print("Número de falhas com duração maior que 1 dia:", count_above_96_60gap)
print(f"Isto corresponde a {(count_above_96_60gap*100)/len(missings_gap60_df):.2f}% das falhas")

In [ ]:
above_96_60gap

In [ ]:
above_192_60gap = missings_gap60_df[missings_gap60_df['Consecutive_Points'] > 192]
count_above_192_60gap = above_192_60gap.shape[0]

print("Número de falhas com duração maior que 2 dias:", count_above_192_60gap)
print(f"Isto corresponde a {(count_above_192_60gap*100)/len(missings_gap60_df):.2f}% das falhas")

In [ ]:
above_192_60gap

In [ ]:
less_than_1day = missings_gap30_df[missings_gap30_df['Consecutive_Points'] < 96]

yticks_labels = [convert_to_time_less_than_1day(y) for y in [0, 20, 40, 60, 80]]

sns.boxplot(data=less_than_1day, whis=100)
plt.yticks([0, 20, 40, 60, 80], yticks_labels)
plt.title("Estudo de Falhas pós normalização com duração menor do que 1 dia")
plt.show()

In [ ]:
if not less_than_1day.empty:
    max_index = less_than_1day['Consecutive_Points'].idxmax()
    row_with_max_points = less_than_1day.loc[max_index]
    print(row_with_max_points)
else:
    print("No rows with Consecutive_Points < 96 found.")

In [ ]:
len(less_than_1day)

In [ ]:
time_intervals = pd.timedelta_range(start='00:00:00', end='23:45:00', freq='5H').astype(str)
time_intervals = time_intervals.insert(len(time_intervals), '24:00:00')

table_data = pd.DataFrame(columns=['Intervalo de Tempo', 'Número de falhas', 'Número de falhas %*'])

for i in range(len(time_intervals) - 1):
    start_time = time_intervals[i].split()[-1]
    end_time = time_intervals[i + 1].split()[-1]

    points_in_interval = less_than_1day[
        (less_than_1day['Consecutive_Points'] >= i * 20) & (less_than_1day['Consecutive_Points'] < (i + 1) * 20)
    ]

    count = len(points_in_interval)
    total_points = len(less_than_1day)
    percentage = (count / total_points) * 100 if total_points > 0 else 0

    table_data.loc[i] = [f'{start_time} <= x < {end_time}', count, f'{percentage:.2f}%']

print(table_data)
print("\n*% referente ao total de falhas com duração menor que 1 dia")

In [ ]:
above_96_30gap
print("Gap de 30 min")
print(f"A percentagem de falhas superior a 1 dia é de {((len(above_96_30gap)*100)/len(missings_gap30_df)):.2f}%")
print(f"A percentagem de falhas inferior a 1 dia é de {((len(less_than_1day)*100)/len(missings_gap30_df)):.2f}%")

In [ ]:
time_intervals = pd.timedelta_range(start='00:00:00', end='23:45:00', freq='2H').astype(str)
time_intervals = time_intervals.insert(len(time_intervals), '24:00:00')

table_data = pd.DataFrame(columns=['Intervalo de Tempo', 'Número de falhas', 'Número de falhas %*'])

for i in range(len(time_intervals) - 1):
    start_time = time_intervals[i].split()[-1]
    end_time = time_intervals[i + 1].split()[-1]

    points_in_interval = less_than_1day[
        (less_than_1day['Consecutive_Points'] >= i * 8) & (less_than_1day['Consecutive_Points'] < (i + 1) * 8)
    ]

    count = len(points_in_interval)
    total_points = len(less_than_1day)
    percentage = (count / total_points) * 100 if total_points > 0 else 0

    table_data.loc[i] = [f'{start_time} <= x < {end_time}', count, f'{percentage:.2f}%']
print(table_data)
print("\n*% referente ao total de falhas com duração menor que 1 dia")

In [ ]:
missing_to_12h = less_than_1day[less_than_1day['Consecutive_Points'] <= 48].copy()
missing_to_12h['Caudal'] = 0
print(statistics(missing_to_12h).drop(['Min_Value', 'Max_Value'], axis=1))

print(f"\nExistem {len(missing_to_12h)} falhas menores que 12h o que corresponde a  {(len(missing_to_12h)*100)/len(max30min_df)}% da amostra")

In [ ]:
missing_from_12h_24h = less_than_1day[less_than_1day['Consecutive_Points'] > 48].copy()
missing_from_12h_24h['Caudal'] = 0

if len(missing_from_12h_24h) > 0:
  print(statistics(missing_from_12h_24h).drop(['Min_Value', 'Max_Value'], axis=1))

print(f"\nExistem {len(missing_from_12h_24h)} falhas menores que 1 dia e maiores do que 12h o que corresponde a  {(len(missing_from_12h_24h)*100)/len(max30min_df)}% da amostra")

In [ ]:
more_than_24h = missings_gap30_df[missings_gap30_df['Consecutive_Points'] > 96].copy()
more_than_24h['Caudal'] = 0

if len(more_than_24h) > 0:
  print(statistics(more_than_24h).drop(['Min_Value', 'Max_Value'], axis=1))

print(f"\nExistem {len(more_than_24h)} falhas maiores que 1 dia o que corresponde a  {(len(more_than_24h)*100)/len(max30min_df)}% da amostra")


In [ ]:
def safe_statistics(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except ValueError:
        return pd.DataFrame(columns=['Year', 'Month', 'Min_Value', 'Max_Value', 'Count'])

stats_df = safe_statistics(statistics, missings_df)
stats_outliers_above = safe_statistics(statistics, outliers_above)
stats_outliers_below = safe_statistics(statistics, outliers_below)
stats_df_normalized = safe_statistics(statistics, missings_normalized)
stats_missing_to_12h = safe_statistics(statistics, missing_to_12h)
stats_missing_from_12h_24h = safe_statistics(statistics, missing_from_12h_24h)
stats_more_than_24h = safe_statistics(statistics, more_than_24h)

In [ ]:
def year_month(df):
  df['Year_Month'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
  df.set_index('Year_Month', inplace=True)

In [ ]:
year_month(stats_df)
year_month(stats_outliers_above)
year_month(stats_outliers_below)
year_month(stats_df_normalized)
year_month(stats_missing_to_12h)
year_month(stats_missing_from_12h_24h)
year_month(stats_more_than_24h)

In [ ]:
def drop_Year_And_Month(df):
  df = df.drop(['Year', 'Month'], axis=1)
  return df

In [ ]:
def drop_Min_And_Max(df):
  df = df.drop(['Min_Value', 'Max_Value'], axis=1)
  return df

In [ ]:
stats_df = drop_Year_And_Month(stats_df)
stats_outliers_above = drop_Year_And_Month(stats_outliers_above)
stats_outliers_below = drop_Year_And_Month(stats_outliers_below)
stats_df_normalized = drop_Year_And_Month(stats_df_normalized)
stats_missing_to_12h = drop_Year_And_Month(stats_missing_to_12h)
stats_missing_from_12h_24h = drop_Year_And_Month(stats_missing_from_12h_24h)
stats_more_than_24h = drop_Year_And_Month(stats_more_than_24h)

In [ ]:
stats_df = drop_Min_And_Max(stats_df)
stats_df_normalized = drop_Min_And_Max(stats_df_normalized)
stats_missing_to_12h = drop_Min_And_Max(stats_missing_to_12h)
stats_missing_from_12h_24h = drop_Min_And_Max(stats_missing_from_12h_24h)
stats_more_than_24h = drop_Min_And_Max(stats_more_than_24h)

In [ ]:
merged_df = pd.DataFrame()

In [ ]:
dfs = [stats_df, stats_outliers_above, stats_outliers_below, stats_df_normalized, stats_missing_to_12h, stats_missing_from_12h_24h, stats_more_than_24h]

merged_df = dfs[0]

for i in range(1, len(dfs)):
    merged_df = merged_df.merge(dfs[i], left_index=True, right_index=True, how='outer', suffixes=('', f'_df{i}'))


In [ ]:
merged_df.index = pd.to_datetime(merged_df.index, format='%Y-%m')
merged_df = merged_df.sort_index()
merged_df.index = merged_df.index.to_period('M').astype(str)

In [ ]:
merged_df = merged_df.rename(columns={
    'Count': 'série bruta registos sem caudal (%)',
    'Min_Value': 'série bruta oultliers de caudal máximo(LI) (m3/h)',
    'Max_Value': 'série bruta oultliers de caudal máximo(LS) (m3/h)',
    'Count_df1': 'série bruta nº de oultliers de caudal máximo (nº)',
    'Min_Value_df2': 'série bruta oultliers de caudal minimo(LI) (m3/h)',
    'Max_Value_df2': 'série bruta oultliers de caudal minimo(LS) (m3/h)',
    'Count_df2': 'série bruta nº de oultliers de caudal minimo (nº)',
    'Count_df3': 'série normalizada valores omissos (%)',
    'Count_df4': 'série normalizada falhas com duração inf. 12 h (n.º)',
    'Count_df5': 'série normalizada falhas com duração 12 - 24 h (n.º)',
    'Count_df6': 'série normalizada falhas com duração sup. 24 h (n.º)'
})

In [ ]:
merged_df['série bruta registos sem caudal (%)'] = (merged_df['série bruta registos sem caudal (%)']*100)/len(df)

In [ ]:
merged_df['série normalizada valores omissos (%)'] = (merged_df['série normalizada valores omissos (%)']*100)/len(max30min_df)

In [ ]:
merged_df

In [ ]:
max30min_df.reset_index(inplace=True)

In [ ]:
max30min_df = max30min_df.drop(['Year', 'Month'], axis=1)

In [ ]:
first_row_time = max30min_df.iloc[0]['Data']
if first_row_time.time() != pd.to_datetime('00:00:00').time():
    start_time = first_row_time.replace(hour=0, minute=0, second=0, microsecond=0)
    end_time = first_row_time

    time_range = pd.date_range(start=start_time, end=end_time, freq='15T')

    time_range = time_range[time_range < end_time]

    new_rows = pd.DataFrame({'Data': time_range, 'Caudal': np.nan})

    max30min_df = pd.concat([new_rows, max30min_df]).reset_index(drop=True)

In [ ]:
max30min_df

In [ ]:
copy1 = max30min_df.copy()

In [ ]:
aux = copy1.values

In [ ]:
matriz = [[None] * 97 for _ in range(1097)]

In [ ]:
matriz[0][0] = 'Date'

In [ ]:
for i in range(96):
  matriz[0][i+1] = aux[i][0].strftime('%H:%M')

In [ ]:
auxlenght = 0
for k in range(1096):
  first = 0
  for j in range(96):
    if first == 0:
      matriz[k+1][0] = aux[auxlenght][0].strftime('%Y/%m/%d')

      if auxlenght + 95 > len(aux):
        break

    auxlenght += 1
    first = 1

In [ ]:
auxlenght = 0

for m in range(1096):
  for n in range(96):
    if auxlenght >= len(aux):
      break

    matriz[m+1][n+1] = aux[auxlenght][1]
    auxlenght += 1

In [ ]:
columns = matriz[0]

data = matriz[1:]

rexcel = pd.DataFrame(data, columns=columns)

In [ ]:
rexcel

In [ ]:
max30min_df.to_excel('normalized.xlsx', index=False)

In [ ]:
writer = pd.ExcelWriter('rexcel.xlsx', engine='xlsxwriter')

rexcel.to_excel(writer, sheet_name='Sheet1', index=False)

workbook  = writer.book
worksheet = writer.sheets['Sheet1']

header_format = workbook.add_format({})

for col_num, value in enumerate(rexcel.columns.values):
    worksheet.write(0, col_num, value, header_format)

writer.close()

#Preenchimento de Falhas

In [ ]:
reconstruct_forecast_Anual = pd.read_csv('data_with_Anual.csv', delimiter=',')

In [ ]:
reconstruct_forecast_Anual.loc[0, 'Data'] = '2013-07-01  00:00:00'

In [ ]:
reconstruct_forecast_Anual.isnull().any().any()

In [ ]:
reconstruct_forecast_Weekly = pd.read_csv('data_with_Weekly.csv', delimiter=',')

In [ ]:
reconstruct_forecast_Weekly.loc[0, 'Data'] = '2013-07-01  00:00:00'

In [ ]:
reconstruct_forecast_Weekly.isnull().any().any()

In [ ]:
reconstruct_Quevedo = pd.read_csv('Quevedo.csv', delimiter=',')

In [ ]:
reconstruct_Quevedo = reconstruct_Quevedo.drop("Unnamed: 0", axis = 1)

In [ ]:
reconstruct_Quevedo.rename(columns=lambda x: x[1:] if x.startswith('X') else x, inplace=True)

In [ ]:
reconstruct_Quevedo.rename(columns=lambda x: x.replace('.', ':'), inplace=True)

In [ ]:
df_long = pd.melt(reconstruct_Quevedo, id_vars=['Date'], var_name='Time', value_name='Caudal')

df_long['DateTime'] = pd.to_datetime(df_long['Date'] + ' ' + df_long['Time'], format='%Y/%m/%d %H:%M')

df_long = df_long.drop(columns=['Date', 'Time'])

df_long = df_long[['DateTime', 'Caudal']]

df_long = df_long.sort_values(by='DateTime').reset_index(drop=True)

In [ ]:
reconstruct_Quevedo = df_long

In [ ]:
reconstruct_Quevedo.isnull().any().any()

In [ ]:
reconstruct_tbats = pd.read_csv('TBATS.csv', delimiter=',')
reconstruct_tbats = reconstruct_tbats.drop("Unnamed: 0", axis = 1)
reconstruct_tbats.rename(columns=lambda x: x[1:] if x.startswith('X') else x, inplace=True)
reconstruct_tbats.rename(columns=lambda x: x.replace('.', ':'), inplace=True)

df_long = pd.melt(reconstruct_tbats, id_vars=['Date'], var_name='Time', value_name='Caudal')

df_long['DateTime'] = pd.to_datetime(df_long['Date'] + ' ' + df_long['Time'], format='%Y/%m/%d %H:%M')

df_long = df_long.drop(columns=['Date', 'Time'])

df_long = df_long[['DateTime', 'Caudal']]

df_long = df_long.sort_values(by='DateTime').reset_index(drop=True)

reconstruct_tbats = df_long

In [ ]:
reconstruct_tbats.isnull().any().any()

In [ ]:
TBATS_nan_mask = reconstruct_tbats.isnull().any(axis=1)

num_nan_rows = TBATS_nan_mask.sum()
print(f"Number of rows with NaN values: {num_nan_rows}")

nan_rows = reconstruct_tbats[TBATS_nan_mask]
print("Rows with NaN values:")
print(nan_rows)

In [ ]:
nan_rows = copy1[copy1.isnull().any(axis=1)]

In [ ]:
reconstructions = pd.DataFrame(columns=['Date', 'forecast_Anual', 'forecast_Weekly', 'Quevedo', 'TBATS'])

for index in nan_rows.index:
    reconstructions.loc[index, 'Date'] = nan_rows.loc[index, 'Data']
    reconstructions.loc[index, 'forecast_Anual'] = reconstruct_forecast_Anual.iloc[index]['Caudal']
    reconstructions.loc[index, 'forecast_Weekly'] = reconstruct_forecast_Weekly.iloc[index]['Caudal']
    reconstructions.loc[index, 'Quevedo'] = reconstruct_Quevedo.iloc[index]['Caudal']
    reconstructions.loc[index, 'TBATS'] = reconstruct_tbats.iloc[index]['Caudal']

In [ ]:
reconstructions

In [ ]:
reconstructions['diff_forecast_Anual'] = ((reconstructions['forecast_Anual'] - reconstructions['TBATS']) / reconstructions['TBATS']) * 100
reconstructions['diff_forecast_Weekly'] = ((reconstructions['forecast_Weekly'] - reconstructions['TBATS']) / reconstructions['TBATS']) * 100
reconstructions['diff_Quevedo'] = ((reconstructions['Quevedo'] - reconstructions['TBATS']) / reconstructions['TBATS']) * 100

reconstructions['diff_forecast_Anual'] = reconstructions['diff_forecast_Anual'].abs()
reconstructions['diff_forecast_Weekly'] = reconstructions['diff_forecast_Weekly'].abs()
reconstructions['diff_Quevedo'] = reconstructions['diff_Quevedo'].abs()


In [ ]:
mean_abs_diff_forecast_Anual = reconstructions['diff_forecast_Anual'].mean()
median_abs_diff_forecast_Anual = reconstructions['diff_forecast_Anual'].median()

mean_abs_diff_forecast_Weekly = reconstructions['diff_forecast_Weekly'].mean()
median_abs_diff_forecast_Weekly = reconstructions['diff_forecast_Weekly'].median()

mean_abs_diff_Quevedo = reconstructions['diff_Quevedo'].mean()
median_abs_diff_Quevedo = reconstructions['diff_Quevedo'].median()

print("Percentagem de diferença de cada método relativamente ao TBATS:")
print(f"Forecast com sazonalidade Anual - Média: {mean_abs_diff_forecast_Anual:.2f}%, Mediana: {median_abs_diff_forecast_Anual:.2f}%")
print(f"Forecast com sazonalidade Semanal - Média: {mean_abs_diff_forecast_Weekly:.2f}%, Mediana: {median_abs_diff_forecast_Weekly:.2f}%")
print(f"Quevedo - Média: {mean_abs_diff_Quevedo:.2f}%, Mediana: {median_abs_diff_Quevedo:.2f}%")

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(reconstructions['Date'], reconstructions['forecast_Anual'], marker='o', linestyle='-', color='blue', label='forecast_Anual')
plt.plot(reconstructions['Date'], reconstructions['forecast_Weekly'], marker='o', linestyle='-', color='red', label='forecast_Weekly')
plt.plot(reconstructions['Date'], reconstructions['Quevedo'], marker='o', linestyle='-', color='green', label='Quevedo')
plt.plot(reconstructions['Date'], reconstructions['TBATS'], marker='o', linestyle='-', color='purple', label='TBATS')

plt.xlabel('Date')
plt.ylabel('Differences')
plt.title('Differences Between Values for Each Forecasting Method Over Time')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(reconstructions['Date'], reconstructions['TBATS'], marker='o', linestyle='-', color='purple', label='TBATS')
plt.plot(reconstructions['Date'], reconstructions['Quevedo'], marker='o', linestyle='-', color='green', label='Quevedo')

plt.xlabel('Date')
plt.ylabel('Differences')
plt.title('Differences Between Values for Each Forecasting Method Over Time')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
reconstructions

In [ ]:
reconstruct_Quevedo

In [ ]:
merged_df

In [ ]:
merged_df.to_excel('results.xlsx', index=True)

#Identificação de Eventos Anómalos

In [ ]:
Avarias_df = pd.read_excel("/content/.xlsx", sheet_name="")

In [ ]:
def datetime_in_range(datetime, start_datetimes, end_datetimes):
    for start, end in zip(start_datetimes, end_datetimes):
        if start <= datetime <= end:
            return True
    return False

In [ ]:
Avarias_df

In [ ]:
original_outlier_above

In [ ]:
original_outlier_below

In [ ]:
original_outlier_above['Data'] = pd.to_datetime(original_outlier_above['Data'])
original_outlier_below['Data'] = pd.to_datetime(original_outlier_below['Data'])
Avarias_df['Data/Hora'] = pd.to_datetime(Avarias_df['Data/Hora'])
Avarias_df['Data Conclusão'] = pd.to_datetime(Avarias_df['Data Conclusão'])

In [ ]:
original_outlier_above['Exists in Avarias_df'] = original_outlier_above['Data'].apply(
    lambda datetime: datetime_in_range(datetime, Avarias_df['Data/Hora'], Avarias_df['Data Conclusão'])
)

original_outlier_below['Exists in Avarias_df'] = original_outlier_below['Data'].apply(
    lambda datetime: datetime_in_range(datetime, Avarias_df['Data/Hora'], Avarias_df['Data Conclusão'])
)

In [ ]:
true_rows_above = original_outlier_above[original_outlier_above['Exists in Avarias_df']]
true_rows_below = original_outlier_below[original_outlier_below['Exists in Avarias_df']]

In [ ]:
combined_true_rows = pd.concat([true_rows_above, true_rows_below])
combined_true_rows.reset_index(drop=True, inplace=True)

In [ ]:
combined_true_rows